In [ ]:
import requests,json,datetime,time
import pandas as pd
from io import StringIO as sio
pd.options.display.float_format = '{:,.2f}'.format
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import plotly.express as px

header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36 Edg/90.0.818.66',
    'Accept': '*/*',
    'content-type': 'application/json',
    'Accept-Encoding': 'gzip, deflate, br',
    }

def getaio(df):
    buyer_stack=df.pivot_table(index="symbol", columns='buyer',values=['qty','amt'],aggfunc={'qty':['sum','count'],'amt': 'sum'}).stack()
    buyer_stack.columns.droplevel()
    buyer_stack.columns='amount count qty'.split()
    buyer_stack.columns='buy_'+buyer_stack.columns
    seller_stack=df.pivot_table(index="symbol", columns='seller',values=['qty','amt'],aggfunc={'qty':['sum','count'],'amt': 'sum'}).stack()
    seller_stack.columns.droplevel()
    seller_stack.columns='amount count qty'.split()
    seller_stack.columns='sale_'+seller_stack.columns
    buy_sale_stack=pd.concat([buyer_stack,seller_stack],axis=1)
    buy_sale_stack.index.names=('symbol','broker')
    buy_sale_stack.fillna(0,inplace=True)
    buy_pct=buy_sale_stack.buy_qty.groupby(level=0).apply(lambda x: 100* x/x.sum())
    sale_pct=buy_sale_stack.sale_qty.groupby(level=0).apply(lambda x: 100* x/x.sum())
    aio=pd.concat([buy_sale_stack,buy_pct,sale_pct],axis=1)
    col_name=list(aio.columns)
    col_name[-2:]=["buy_pct",'sale_pct']
    aio.columns=col_name
    aio.fillna(0,inplace=True)
    return aio
def get_fs(url):
    df=pd.read_csv(sio(requests.get(url).text),thousands=',',low_memory=False)
    df=df["TransactionNumber Symbol	BuyerBrokerCode	SellerBrokerCode	Quantity	Rate	Amount".split()]
    df.columns="contract symbol buyer seller qty rate amt".split()
    df.contract=df.contract.astype(str)
    df.buyer=df.buyer.astype(str)
    df.seller=df.seller.astype(str)
    return df

def latest_n_days(df,n):
  unk=df.index.unique(level='date').sort_values(ascending=False)
  return df.loc[unk[:n].sort_values()]

chart_provider={
'nepsechart':'https://ohlcv.nepsechart.com/history?symbol={symbol}&resolution={resolution}&from={fromtime}&to={totime}',
'merocapital':'https://chartdata.merocapital.com/datafeed1/history?symbol={symbol}&resolution={resolution}&from={fromtime}&to={totime}',
'merolaganida':'https://da.merolagani.com/handlers/TechnicalChartHandler.ashx?type=get_advanced_chart&symbol={symbol}&resolution={resolution}&rangeStartDate={fromtime}&rangeEndDate={totime}&from=&isAdjust=1&currencyCode=NPR',
'merolagani':'https://www.merolagani.com/handlers/TechnicalChartHandler.ashx?type=get_advanced_chart&symbol={symbol}&resolution={resolution}&rangeStartDate={fromtime}&rangeEndDate={totime}&from=&isAdjust=1&currencyCode=NPR',
'nepsealpha':'https://nepsealpha.com/trading/1/history?symbol={symbol}&resolution={resolution}&from={fromtime}&to={totime}&currencyCode=NRS',
'nepsedata':'https://nepsedata.com/history?symbol={symbol}&resolution={resolution}&from={fromtime}&to={totime}&currencyCode=NRS'
}
def chart_data(provider='merolagani',symbol="NEPSE",fromtime=datetime.date(2019,1,1),totime=datetime.datetime.now(),resolution="1D"):
    """returns data fetched from nepsealpha,merocapital,merolagani and nepsechart.\nwarning!! \nNepsechart,merocapital takes D for daily resolution.\nMerocapital,nepsedata gives unadjusted chart)"""
    fromtime =int(time.mktime(fromtime.timetuple()))
    totime =int(time.mktime(totime.timetuple())) 
    url=chart_provider.get(provider)
    df=pd.read_json(requests.get(url.format(symbol=symbol,resolution=resolution,fromtime=fromtime,totime=totime),headers=header).content)
    df['t']=df['t'].apply(datetime.datetime.utcfromtimestamp)
    df.drop('s',axis=1,inplace=True)
    return df

In [ ]:
def get_madhuko_fs(dater):
  url="https://raw.githubusercontent.com/madhuko/temp/main/fs/{}".format(dater)
  df=pd.read_csv(url,low_memory=False)
  df.contract=df.contract.astype(str)
  df.buyer=df.buyer.astype(str)
  df.seller=df.seller.astype(str)
  df.qty=df.qty.astype(float)
  df.rate=df.rate.astype(float)
  df.amt=df.amt.astype(float)
  contract_1=df.contract[1]
  if dater!=datetime.date(int(contract_1[:4]),int(contract_1[4:6]),int(contract_1[6:8])):
    print("date valdation failed for {}".format(dater))
  return df
  

def getfs_nepsealpha(symbol,s):
  fs=s.get("https://nepsealpha.com/floorsheet_ajx/{}/index".format(symbol))
  new_df=pd.read_html(sio(fs.json()['html']))
  new_df[1]["Symbol"]=symbol
  return new_df[1]

def get_latest_fs():
    s=requests.Session()
    s.headers.update(header)
    s.get("https://nepsealpha.com/trading/chart")
    df=pd.DataFrame()
    live_market=pd.read_html("https://www.merolagani.com/LatestMarket.aspx")[0]
    for sym in live_market["Symbol"]:
        if "/" in sym:
            continue
        try:
          ram=getfs_nepsealpha(sym,s)
        except:
          s=requests.Session()
          s.headers.update(header)
          s.get("https://nepsealpha.com/trading/chart")
          ram=getfs_nepsealpha(sym,s)
        df=pd.concat([df,ram])
        print("Collected data of {}".format(sym))
    df.columns='contract buyer seller qty rate amt symbol'.split()
    df["amt"]=df["amt"].apply(lambda x: x.replace("NPR",""))
    df["amt"]=df["amt"].apply(lambda x: x.replace(",",""))
    df.amt=df.amt.astype(float)
    df.buyer=df.buyer.astype(str)
    df.seller=df.seller.astype(str)
    df["rate"]=df["rate"].apply(lambda x: x.replace("NPR ",""))
    return df

In [ ]:
init_ohlc=chart_data(fromtime=datetime.date(2021,8,1),totime=datetime.date(2021,8,10))
init_ohlc.t=init_ohlc.t.dt.date
aio=pd.DataFrame()
no_of_days=len(init_ohlc)

# get d1 data
dater0=init_ohlc.t.iloc[-1]
try:
    df=get_madhuko_fs(dater0)
except:
    df=get_latest_fs()
taio=getaio(df)
taio['date']=dater0
taio=taio.reset_index().set_index(['date','symbol','broker'])
aio=pd.concat([aio,taio])
df.to_csv("fs/"+str(dater),index=False)
print(dater0)

for i in range(1,no_of_days):
  dater=init_ohlc.t[no_of_days-1-i]
  if dater<datetime.date(2021,8,1):
      df=get_fs("https://datasets.sheezh.com/static/datasets/floorsheets/{}.csv".format(dater))
  else:
      df=get_madhuko_fs(dater)
  taio=getaio(df)
  taio['date']=dater
  taio=taio.reset_index().set_index(['date','symbol','broker'])
  aio=pd.concat([aio,taio])
  print(dater)

In [ ]:
taio=aio.xs(dater0)

In [ ]:
print("TOP turnover")
taio.groupby("symbol").sum().sort_values("buy_amount",ascending=False).head(10)

In [ ]:
# change sort_values('buy_amount',...) by appropriate column header
taio.groupby("broker").sum().sort_values('sale_amount',ascending=False).head(10)

In [ ]:
# brokersiwse filter|| sort by appropriate column header
(taio.xs("58",level='broker')
# .query("sale_count >= 20")
.sort_values('buy_pct',ascending=False).head(10))

In [ ]:
# stockwise filter|| sort by appropriate column header
taio.xs("NTC",level='symbol').sort_values('buy_pct',ascending=False).head()

In [ ]:
print("if your desired broker has frequently made cornering !")
broker_id="58" #type broker id here
threshhold=20 # threshhold level of buy or sale
latest_data=10 #how many latests data you want to use
count=2 #minimum how many times it should have been done

temp=aio.xs(broker_id,level=2)
temp=latest_n_days(temp,latest_data)
temp=temp[temp['sale_pct']>=threshhold]
cor_raw=temp[temp.groupby(["symbol"]).count()>=count].dropna().sort_values(["symbol",'date'],ascending=[True,False])
cor_raw

In [ ]:
cor_raw.groupby("symbol").agg(
    {
        # "sum"
       "buy_amount":"sum","buy_count":"sum","buy_qty":"sum","sale_amount":"sum",'sale_count':"sum","sale_qty":"sum","sale_pct":"sum"
    }
).sort_values("sale_amount", ascending=False)

In [ ]:
print("Top accumulator and seller of your script chartwise")
script="NTC" #stock symbol here
latest_data=no_of_days #how many latests data you want to use|| use 'no_of_days' without quote for all

temp=aio.xs(script,level='symbol')
temp=latest_n_days(temp,latest_data)
temp2=temp.pivot_table(index="broker",values=["buy_qty","sale_qty"],aggfunc=sum)
temp2["net"]=temp2.buy_qty-temp2.sale_qty
temp2.sort_values("net",ascending=False)
temp3=pd.concat([temp2.sort_values("net",ascending=False).head(),temp2.sort_values("net",ascending=False).tail()])
temp3.net.plot.bar()

In [ ]:
broker_id="58"
script="NABIL" #stock symbol here
latest_data=no_of_days #how many latests data you want to use|| use no_of_days for all
def daily_net_position_chart(broker_id=broker_id,script=script,latest_data=latest_data,df=aio):
    temp=latest_n_days(df,latest_data)
    temp=temp.xs(broker_id,level=2)
    temp=temp.xs(script,level=1)
    temp['net']=(temp.buy_qty-temp.sale_qty)/1
    temp.sort_values("date",inplace=True)
    temp.net.plot.line()
    temp.net.cumsum().plot()
    return temp
plt.figure(figsize=(16,6))
ram=daily_net_position_chart()

In [ ]:
broker_id="58" #stock symbol here
latest_data=1 #how many latests data you want to use|| use very high number for all
temp=latest_n_days(aio,latest_data)
temp=temp.xs(broker_id,level='broker')
temp['net']=temp.buy_amount-temp.sale_amount
temp2=temp.groupby("symbol").sum()
temp2["absnet"]=temp2.net.apply(abs)
temp2.sort_values('absnet',ascending=False,inplace=True)
temp2.head(10).net.plot.bar()

In [ ]:
url='https://newweb.nepalstock.com.np/api/nots/company/list'
resp=requests.get(url,headers=header).json()
symbol_sect={}
for i in resp:
    symbol_sect[i['symbol']]=i['sectorName']
def attach_sector(df):
    index=aio.index.names
    df["sector"]=df.symbol.apply(lambda x: symbol_sect.get(x))
    return df
def get_public_share(id):
    # return id
    reply=requests.post("https://newweb.nepalstock.com/api/nots/security/"+str(id),data='{"id":758}',headers=header).json()
    return reply['publicShares']
def get_ps(symbol):
   return cpt[cpt["symbol"]==symbol]["ps"][0] 

def get_itype(symbol):
    return cpt[cpt["symbol"]==symbol]["instrumentType"][0] 

cpt=pd.read_json(sio(json.dumps(resp)))
aio=attach_sector(aio.reset_index())
aio["type"]=aio.symbol.apply(get_itype )
aio.set_index("date symbol broker".split(),inplace=True)


In [ ]:
ns=requests.Session()
ns.headers.update(header)

In [ ]:
resp=ns.get("https://newweb.nepalstock.com/api/authenticate/prove")

In [ ]:
def get_sheezh_fs(dater):
    url="https://datasets.sheezh.com/static/datasets/floorsheets/{}.csv".format(dater)
    df=get_fs(url)
    taio=getaio(df)
    taio['date']=dater
    taio=taio.reset_index().set_index(['date','symbol','broker'])
    aio=pd.concat([aio,taio])
    print(dater)

In [ ]:
"https://datasets.sheezh.com/static/datasets/floorsheets/{}.csv".format(dater)